# Imports & Prepare Data

In [1]:
from models.ml_sequential import prepare_data, run_all
import tensorflow as tf
# Disable GPU training, comment out to enable
tf.config.set_visible_devices([], 'GPU')
# Imports used across models
from keras.layers import Input, Dense, TimeDistributed, Dropout
from keras.models import Sequential
from keras.regularizers import l2

features = ['female', 'age', 'height', 'mass', 'ta_set', 'rh_set']
features_scaler, output_scaler, X_padded_folds, y_padded_folds, max_len = prepare_data(features)

# Define the input/output shape
input_shape = (None, X_padded_folds[0].shape[-1])
output_shape = y_padded_folds[0].shape[-1]

# Train the model with early stopping
from keras.callbacks import EarlyStopping

# Custom log callback
class PrintCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f'Epoch: {epoch+1}, loss: {logs["loss"]:.2f}', end='\r')

    def on_train_end(self, epoch):
        print()

Max sequence length: 660
Max sequence length: 660
Max sequence length: 660
Max sequence length: 660


# RNN

In [ ]:
from keras.layers import SimpleRNN

# MODEL NAME
model_name = 'ml_rnn'
models = []

for idx, (X_padded, y_padded) in enumerate(zip(X_padded_folds, y_padded_folds)):
	fold_number = idx + 1
	print("Fold:", fold_number)
	# Model architecture
	model = Sequential()
	model.add(Input(shape=input_shape))
	model.add(SimpleRNN(16, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01)))
	model.add(TimeDistributed(Dense(8, activation='linear', kernel_regularizer=l2(0.01))))
	model.add(Dropout(0.2))
	model.add(TimeDistributed(Dense(output_shape)))

	# Compile the model
	model.compile(loss='mse', optimizer='adam')

	# Train the model
	early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
	model.fit(X_padded, y_padded, validation_split=0.2, epochs=500, batch_size=32, callbacks=[PrintCallback(), early_stopping], verbose=0)

	# Save the model
	model.save('model_weights/{}-fold{}.keras'.format(model_name, fold_number))

	models.append(model)

# Run simulations using trained model
run_all(models, model_name, features, features_scaler, output_scaler, max_len)

Fold: 1
Epoch: 398, loss: 0.04
Fold: 2
Epoch: 360, loss: 0.04
Fold: 3
Epoch: 500, loss: 0.05
Fold: 4
Epoch: 500, loss: 0.04
ml_rnn
Average TRE RMSE: 0.40913350652486097
Average MTSK RMSE: 1.2484756718098815


# LSTM

In [4]:
from keras.layers import LSTM

# MODEL NAME
model_name = 'ml_lstm'
models = []

for idx, (X_padded, y_padded) in enumerate(zip(X_padded_folds, y_padded_folds)):
	fold_number = idx + 1
	print("Fold:", fold_number)
	# Model architecture
	model = Sequential()
	model.add(Input(shape=input_shape))
	model.add(LSTM(16, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01)))
	model.add(TimeDistributed(Dense(8, activation='linear', kernel_regularizer=l2(0.01))))
	model.add(Dropout(0.2))
	model.add(TimeDistributed(Dense(output_shape)))

	# Compile the model
	model.compile(loss='mse', optimizer='adam')

	# Train the model
	early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
	model.fit(X_padded, y_padded, validation_split=0.2, epochs=500, batch_size=32, callbacks=[PrintCallback(), early_stopping], verbose=0)

	# Save the model
	model.save('model_weights/{}-fold{}.keras'.format(model_name, fold_number))

	models.append(model)

# Run simulations using trained model
run_all(models, model_name, features, features_scaler, output_scaler, max_len)

Fold: 1
Epoch: 282, loss: 0.04
Fold: 2
Epoch: 296, loss: 0.04
Fold: 3
Epoch: 285, loss: 0.04
Fold: 4
Epoch: 355, loss: 0.04
ml_lstm
Average TRE RMSE: 0.4006921834540827
Average MTSK RMSE: 1.1933293403769494


# GRU

In [5]:
from keras.layers import GRU

# MODEL NAME
model_name = 'ml_gru'
models = []

for idx, (X_padded, y_padded) in enumerate(zip(X_padded_folds, y_padded_folds)):
	fold_number = idx + 1
	print("Fold:", fold_number)
	# Model architecture
	model = Sequential()
	model.add(Input(shape=input_shape))
	model.add(GRU(16, return_sequences=True))
	model.add(TimeDistributed(Dense(8, activation='linear', kernel_regularizer=l2(0.01))))
	model.add(Dropout(0.2))
	model.add(TimeDistributed(Dense(output_shape)))

	# Compile the model
	model.compile(loss='mse', optimizer='adam')

	# Train the model
	early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
	model.fit(X_padded, y_padded, validation_split=0.2, epochs=500, batch_size=32, callbacks=[PrintCallback(), early_stopping], verbose=0)

	# Save the model
	model.save('model_weights/{}-fold{}.keras'.format(model_name, fold_number))

	models.append(model)

# Run simulations using trained model
run_all(models, model_name, features, features_scaler, output_scaler, max_len)

Fold: 1
Epoch: 255, loss: 0.02
Fold: 2
Epoch: 322, loss: 0.02
Fold: 3
Epoch: 425, loss: 0.01
Fold: 4
Epoch: 313, loss: 0.01
ml_gru
Average TRE RMSE: 0.35102718032762875
Average MTSK RMSE: 0.9882427786586373


# TCN

In [10]:
# from tensorflow.keras.layers import Conv1D

# # MODEL NAME
# model_name = 'ml_tcn'

# model = Sequential()
# model.add(Input(shape=input_shape))
# model.add(Conv1D(16, kernel_size=4, activation='linear', padding='causal', kernel_regularizer=l2(0.01)))
# model.add(TimeDistributed(Dense(8, activation='linear', kernel_regularizer=l2(0.01))))
# model.add(Dropout(0.2))
# model.add(TimeDistributed(Dense(output_shape)))

# model.compile(loss='mse', optimizer='adam')

# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
# model.fit(X_padded, y_padded, validation_split=0.2, epochs=500, batch_size=32, callbacks=[PrintCallback(), early_stopping], verbose=0)

# model.save('model_weights/{}.keras'.format(model_name))

# # Run simulations using trained model
# run_all(model, model_name, features, features_scaler, output_scaler, max_len)

Epoch: 500, loss: 0.03
ml_tcn
Average TRE RMSE: 0.4642991150064746
Average MTSK RMSE: 1.6289565766153424


# Seq2Seq

In [7]:
# from tensorflow.keras.layers import LSTM, Attention, Concatenate
# from tensorflow.keras.models import Model

# # MODEL NAME
# model_name = 'ml_seq2seq'

# encoder_inputs = Input(shape=input_shape)
# encoder_lstm = LSTM(16, return_sequences=True, return_state=True, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))
# encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# decoder_inputs = Input(shape=input_shape)
# decoder_lstm = LSTM(16, return_sequences=True, return_state=True, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))
# decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=[state_h, state_c])

# attention = Attention()([decoder_outputs, encoder_outputs])
# decoder_concat = Concatenate()([decoder_outputs, attention])
# decoder_dense = TimeDistributed(Dense(output_shape))
# decoder_outputs = decoder_dense(decoder_concat)

# model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# model.compile(optimizer='adam', loss='mse')

# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
# model.fit([X_padded, X_padded], y_padded, validation_split=0.2, epochs=500, batch_size=32, callbacks=[PrintCallback(), early_stopping], verbose=0)

# model.save('model_weights/{}.keras'.format(model_name))

# # Run simulations using trained model
# run_all(model, model_name, features, features_scaler, output_scaler, max_len, is_transformer=True)

Epoch: 185, loss: 0.03
ml_seq2seq
Average TRE RMSE: 0.4447912235248274
Average MTSK RMSE: 1.4460834447802842
